# Julia 深度學習：可微分程式設計

本範例需要使用到的套件有 Zygote，請在執行以下範例前先安裝。

```
] add Zygote
```

In [1]:
using Zygote

┌ Info: Precompiling Zygote [e88e6eb3-aa80-5325-afca-941959d7151f]
└ @ Base loading.jl:1260


## 單純函式

In [2]:
f(x) = x^2 + 2x + 1

f (generic function with 1 method)

In [3]:
f(5)

36

`gradient` 是用來求微分值的函式，第一個參數為函式，第二個參數為求微分值的點。

In [4]:
gradient(f, 5)

(12,)

也可以透過在函式名稱後加上 `'` 來取得微分值，這會比較接近一般數學的寫法。

In [5]:
f'(5)

12

In [6]:
g(x) = sin(x)

g (generic function with 1 method)

In [7]:
g'(π)

-1.0

In [8]:
gradient(log, 2)

(0.5,)

In [9]:
gradient(exp, 2)

(7.38905609893065,)

In [10]:
f(x) = log(1 + exp(5*sin(x) + 5x + 2))

f (generic function with 1 method)

In [11]:
f'(1)

7.701406990501207

## 多參數函式

In [12]:
g(a, b) = a*b

g (generic function with 2 methods)

多參數函式需要同時給予，對應求微分值的點，例如這邊示範兩個參數，就需要給兩個點。這時候它回傳的結果，分別是對第一個參數微分 $\frac{\partial g}{\partial a}$ 及第二個參數微分 $\frac{\partial g}{\partial b}$。

In [13]:
gradient(g, 3, 4)

(4, 3)

## 一般函式的微分

In [14]:
function foo(x, n)
    if n == 0
        return x
    end
    return foo(x*n, n-1)
end

foo (generic function with 1 method)

In [18]:
foo(5, 4)

120

In [19]:
gradient(x->foo(x, 4), 5)

(24,)

## 察看微分後函式的 LLVM IR

In [20]:
f(x) = 5*sin(x)

f (generic function with 1 method)

In [21]:
@code_llvm f'(π)


;  @ C:\Users\qwerz\.julia\packages\Zygote\YeCEW\src\compiler\interface.jl:58 within `#38'
; Function Attrs: uwtable
define double @"julia_#38_19714"() #0 {
top:
  ret double -5.000000e+00
}


## 對資料結構微分

In [23]:
d = Dict(:a=>sin, :b=>cos)

Dict{Symbol,Function} with 2 entries:
  :a => sin
  :b => cos

In [24]:
gradient(d[:a], 5)

(0.28366218546322625,)

## 對自訂的型別微分

In [25]:
import Base: +, -

struct Point
  x::Float64
  y::Float64
end

In [26]:
+(a::Point, b::Point) = Point(a.x + b.x, a.y + b.y)

+ (generic function with 208 methods)

In [27]:
a = Point(1, 2)
b = Point(3, 4)

Point(3.0, 4.0)

In [28]:
gradient(y->(y+b).x, a)

((x = 1.0, y = nothing),)